In [1]:
import zipfile
from glob import glob
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model, load_model, save_model
import warnings
warnings.filterwarnings('ignore')
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tensorflow.image import resize
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import re
from keras_unet_collection import models, losses
from keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.metrics import Accuracy
from skimage.metrics import structural_similarity as ssim
from tensorflow.keras.layers import Input, BatchNormalization, Dropout, Lambda,add, Conv2D, Conv2DTranspose, MaxPooling2D
from tensorflow.keras.layers import UpSampling2D, concatenate, BatchNormalization, Activation, SeparableConv2D

In [2]:
data_path = "G:\\Harsh_Birva_minor_Proj\\FIVES\\train"


retina_files = glob(pathname = data_path+'\\Original\\*')
retina_files[:1]

mask_files = [path.replace('Original', 'Ground truth') for path in retina_files]
mask_files[:1]

data_path_test = "G:\\Harsh_Birva_minor_Proj\\FIVES\\test"

retina_files_test = glob(pathname = data_path_test+'\\Original\\*')
retina_files_test[:1]

mask_files_test = [path.replace('Original', 'Ground truth') for path in retina_files_test]
mask_files_test[:1]

df_train = pd.DataFrame( data = {
    'retina': retina_files,
    'mask': mask_files
})

df_test = pd.DataFrame( data = {
    'retina': retina_files_test,
    'mask': mask_files_test
})

for i,v in enumerate(df_train['retina']):
    if(v[-3:]!='png'):
        print(i)
        drop_row = i
        break

df_train.drop(drop_row, inplace=True)

600


In [3]:
def load_and_preprocess_image(file_path,resize):
    img = cv2.imread(file_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img_rgb, (resize,resize))
    img_scaled = img_resized.astype(np.float32) / 255.0
    return img_scaled

In [4]:
def load_and_preprocess_mask(file_path,resize):
    img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE )
    img = img[:,:,np.newaxis]
    img_resized = cv2.resize(img, (resize,resize))
    img_scaled = img_resized.astype(np.float32) / 255.0
    img_scaled[img_scaled >= 0.25] = 1
    img_scaled[img_scaled < 0.25] = 0
    return img_scaled

In [5]:
import IMAGE_DATA_AUGMENTATION_LIB as IDAL
def augment_batch(imgs, masks):    
    for i  in range(imgs.shape[0]):
        imgs[i], masks[i] = IDAL.Augment_RandomAffine_2D(imgs[i], masks[i],
                                     translation=0.05, 
                                     rotation_range=[-10,10], 
                                     scale_range=[0.95, 1.05], 
                                     interpolation=cv2.INTER_NEAREST)
    return imgs, masks

In [6]:
def generate_data_generator(dataframe,resize, batch_size=8):
    num_samples = len(dataframe)
    while True:
        indices = np.random.randint(0, num_samples, batch_size)
        batch_retina = []
        batch_mask = []
        for idx in indices:
            retina_img = load_and_preprocess_image(dataframe.iloc[idx]['retina'],resize)
            mask_img = load_and_preprocess_mask(dataframe.iloc[idx]['mask'],resize)
            batch_retina.append(retina_img)
            batch_mask.append(mask_img)
        batch_retina = np.array(batch_retina)
        batch_mask = np.array(batch_mask)
        batch_retina, batch_mask = augment_batch(batch_retina, batch_mask)
        
        yield batch_retina, batch_mask

In [7]:
batch_size = 8
train_generator = generate_data_generator(df_train, batch_size=batch_size,resize=512)
test_generator = generate_data_generator(df_test, batch_size=batch_size,resize=512)

In [8]:
def dice_coeff(y_true, y_pred, smooth=100):
    y_true_flatten = K.flatten(y_true)
    y_pred_flatten = K.flatten(y_pred)

    intersection = K.sum(y_true_flatten * y_pred_flatten)
    union = K.sum(y_true_flatten) + K.sum(y_pred_flatten)
    return (2 * intersection + smooth) / (union + smooth)

def dice_coeff_loss(y_true, y_pred, smooth=100):
    return 1 - dice_coeff(y_true, y_pred, smooth)

In [9]:
def DICE_COE(mask1, mask2):
    intersect = np.sum(mask1*mask2)
    fsum = np.sum(mask1)
    ssum = np.sum(mask2)
    dice = (2 * intersect ) / (fsum + ssum)
    dice = np.mean(dice)
    dice = round(dice, 3)
    return dice    

In [10]:
kernel = np.array([[0,1,0],[1,1,1],[0,1,0]],dtype=np.uint8)

In [11]:
def evaluate(model,resize,full=None,thin=None,thick=None):
    if full is None:
        full = []
    if thin is None:
        thin = []
    if thick is None:
        thick = []
        
    for i in range(200):
        retina_img = load_and_preprocess_image(df_test.iloc[i]['retina'],resize)
        mask_img = load_and_preprocess_mask(df_test.iloc[i]['mask'],resize)
        
        retina_img = retina_img[np.newaxis,:,:,:]
        pred_img = model.predict(retina_img)
        pred_img = (np.squeeze(pred_img) > 0.5).astype(np.uint8)
        mask_img = np.squeeze(mask_img).astype(np.uint8)

        full.append(DICE_COE(mask_img,pred_img))

        original = mask_img.copy()
        original_pred = pred_img.copy()

        pred_img = pred_img.astype(float)
        mask_img = mask_img.astype(float)

        for i in range(5):
            mask_img = cv2.erode(mask_img, kernel)
            pred_img = cv2.erode(pred_img, kernel)

            mask_img = cv2.dilate(mask_img, kernel)
            pred_img = cv2.dilate(pred_img, kernel)

        pred_img = pred_img.astype(np.uint8)
        mask_img = mask_img.astype(np.uint8)

        original_thin = cv2.subtract(original,mask_img).astype(np.uint8)
        predicted_thin = cv2.subtract(original_pred,pred_img).astype(np.uint8)

        thin.append(DICE_COE(original_thin,predicted_thin))
        thick.append(DICE_COE(mask_img, pred_img))
        
    return full,thin,thick

In [ ]:
def block(inputs, filters):
    conv = Conv2D(filters, kernel_size = (3,3), padding="same")(inputs)
    bn = Activation("relu")(conv)
    conv = Conv2D(filters, kernel_size = (3,3), padding="same")(bn)
    bn = Activation("relu")(conv)

    return bn

In [ ]:
model_atten_8 = models.att_unet_2d((None,None,3), filter_num = [16,16,16,16],
               n_labels = 1,
               stack_num_down = 2, stack_num_up = 2,
               activation = 'ReLU',
               output_activation = 'Sigmoid',
               atten_activation='ReLU', attention='add',
               pool=True, unpool=True,
               batch_norm = True)

In [ ]:
model_atten_8.summary()

In [ ]:
mynet_16 = my_net_16(input_shape = (None,None,3))
mynet_16.summary()

# MyNet (Filters=16)

In [ ]:
f = 16
num_blocks = 8

In [ ]:
def my_net_16(input_shape = (None, None, 3)):
    inputs = Input(input_shape)
    x = Conv2D(filters = f, kernel_size = (3,3), padding = 'same', activation = 'relu')(inputs)
    x = BatchNormalization()(x)
    
    for i in range(num_blocks):
        x = block(x,f)
        
        if i == 3 or i == 7:
            x = BatchNormalization()(x)
            b = Conv2D(filters = f, kernel_size = (3,3), padding = 'same', activation = 'relu')(inputs)
            b = BatchNormalization()(b)
            x = add([x, b])
  
    outputs = Conv2D(filters = 1, kernel_size = (1,1), activation = "sigmoid")(x)

    return Model(inputs=[inputs], outputs = [outputs])

In [ ]:
epochs=50

In [ ]:
for i in range(5):
    filepath = f"G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\mynet_16-{i+1}.keras"

    checkpoint = ModelCheckpoint(filepath, 
                             verbose=1 
                             )
    
    mynet_16 = my_net_16(input_shape = (None,None,3))
    mynet_16.compile(optimizer='adam', loss=dice_coeff_loss, metrics=[dice_coeff])
    
    history = mynet_16.fit(
        train_generator,
        steps_per_epoch=len(df_train) // batch_size,
        epochs=epochs,
        callbacks=[checkpoint],
        validation_data=test_generator,
        validation_steps=len(df_test) // batch_size
    )
    
    mynet_16_dice,mynet_16_thin_dice,mynet_16_thick_dice = evaluate(mynet_16,512,None,None,None)

In [ ]:
mynet_16_dice

In [ ]:
mynet_16_thin_dice

In [ ]:
mynet_16_thick_dice

# MyNet (Filters=8)

In [ ]:
f = 8
num_blocks = 8

In [ ]:
def my_net_8(input_shape = (None, None, 3)):
    inputs = Input(input_shape)
    x = Conv2D(filters = f, kernel_size = (3,3), padding = 'same', activation = 'relu')(inputs)
    x = BatchNormalization()(x)
    
    for i in range(num_blocks):
        x = block(x,f)
        
        if i == 3 or i == 7:
            x = BatchNormalization()(x)
            b = Conv2D(filters = f, kernel_size = (3,3), padding = 'same', activation = 'relu')(inputs)
            b = BatchNormalization()(b)
            x = add([x, b])
  
    outputs = Conv2D(filters = 1, kernel_size = (1,1), activation = "sigmoid")(x)

    return Model(inputs=[inputs], outputs = [outputs])

In [ ]:
epochs=50

In [ ]:
for i in range(5):
    filepath = f"G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\mynet_8-{i+1}.keras"

    checkpoint = ModelCheckpoint(filepath, 
                             verbose=1 
                             )
    
    mynet_8 = my_net_8(input_shape = (None,None,3))
    mynet_8.compile(optimizer='adam', loss=dice_coeff_loss, metrics=[dice_coeff])
    
    history = mynet_8.fit(
        train_generator,
        steps_per_epoch=len(df_train) // batch_size,
        epochs=epochs,
        callbacks=[checkpoint],
        validation_data=test_generator,
        validation_steps=len(df_test) // batch_size
    )
    
    mynet_8_dice,mynet_8_thin_dice,mynet_8_thick_dice = evaluate(mynet_8,512,None,None,None)

In [ ]:
mynet_8_dice

In [ ]:
mynet_8_thin_dice

In [ ]:
mynet_8_thick_dice

# Attention U-net (Filters = 16)

In [ ]:
epochs = 50

In [ ]:
for i in range(5):
    model_atten_16 = models.att_unet_2d((None,None,3), filter_num = [16,32,64,128],
                                       n_labels = 1,
                                       stack_num_down = 2, stack_num_up = 2,
                                       activation = 'ReLU',
                                       output_activation = 'Sigmoid',
                                       atten_activation='ReLU', attention='add',
                                       pool=True, unpool=True,
                                       batch_norm = True)
    
    filepath = f"G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\atten_unet_16-{i+1}.keras"

    checkpoint = ModelCheckpoint(filepath, 
                             verbose=1 
                             )
    
    model_atten_16.compile(optimizer='adam', loss=dice_coeff_loss, metrics=[dice_coeff])
    
    history = model_atten_16.fit(
        train_generator,
        steps_per_epoch=len(df_train) // batch_size,
        epochs=epochs,
        callbacks=[checkpoint],
        validation_data=test_generator,
        validation_steps=len(df_test) // batch_size
    )
    
    model_atten_16_dice,model_atten_16_thin_dice,model_atten_16_thick_dice = evaluate(model_atten_16,512,None,None,None)

In [ ]:
model_atten_16_dice

In [ ]:
model_atten_16_thin_dice

In [ ]:
model_atten_16_thick_dice

# Attention U-net (Filters = 8)

In [ ]:
epochs=50

In [ ]:
model_atten_8_dice = []

In [ ]:
for i in range(5):
    model_atten_8 = models.att_unet_2d((None,None,3), filter_num = [8,16,32,64],
               n_labels = 1,
               stack_num_down = 2, stack_num_up = 2,
               activation = 'ReLU',
               output_activation = 'Sigmoid',
               atten_activation='ReLU', attention='add',
               pool=True, unpool=True,
               batch_norm = True)
    
    filepath = f"G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\atten_unet_8-{i+1}.keras"

    checkpoint = ModelCheckpoint(filepath, 
                             verbose=1 
                             )
    
    model_atten_8.compile(optimizer='adam', loss=dice_coeff_loss, metrics=[dice_coeff])
    
    history = model_atten_8.fit(
        train_generator,
        steps_per_epoch=len(df_train) // batch_size,
        epochs=epochs,
        callbacks=[checkpoint],
        validation_data=test_generator,
        validation_steps=len(df_test) // batch_size
    )
    
    model_atten_8_dice,model_atten_8_thin_dice,model_atten_8_thick_dice = evaluate(model_atten_8,512,None,None,None)

In [ ]:
model_atten_8_dice

In [ ]:
model_atten_8_thin_dice

In [ ]:
model_atten_8_thick_dice

# Simple Net (Filters=16)

In [ ]:
f = 16
num_blocks = 8

In [ ]:
def simple_net_16(input_shape = (None, None, 3)):
    inputs = Input(input_shape)
    x = Conv2D(filters = f, kernel_size = (3,3), padding = 'same', activation = 'relu')(inputs)
    x = BatchNormalization()(x)
    
    for i in range(num_blocks):
        x = block(x,f)
        
        if i == 3 or i == 7:
            x = BatchNormalization()(x)
  
    outputs = Conv2D(filters = 1, kernel_size = (1,1), activation = "sigmoid")(x)

    return Model(inputs=[inputs], outputs = [outputs])

In [ ]:
epochs=50

In [ ]:
for i in range(5):
    filepath = f"G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\simplenet_16-{i+1}.keras"

    checkpoint = ModelCheckpoint(filepath, 
                             verbose=1 
                             )
    
    simplenet_16 = simple_net_16(input_shape = (None,None,3))
    simplenet_16.compile(optimizer='adam', loss=dice_coeff_loss, metrics=[dice_coeff])
    
    history = simplenet_16.fit(
        train_generator,
        steps_per_epoch=len(df_train) // batch_size,
        epochs=epochs,
        callbacks=[checkpoint],
        validation_data=test_generator,
        validation_steps=len(df_test) // batch_size
    )
    
    simplenet_16_dice,simplenet_16_thin_dice,simplenet_16_thick_dice = evaluate(simplenet_16,512,None,None,None)

In [ ]:
simplenet_16_dice

In [ ]:
simplenet_16_thin_dice

In [ ]:
simplenet_16_thick_dice

# Simple Net (Filters=8)

In [ ]:
f = 8
num_blocks = 8

In [ ]:
def simple_net_8(input_shape = (None, None, 3)):
    inputs = Input(input_shape)
    x = Conv2D(filters = f, kernel_size = (3,3), padding = 'same', activation = 'relu')(inputs)
    x = BatchNormalization()(x)
    
    for i in range(num_blocks):
        x = block(x,f)
        
        if i == 3 or i == 7:
            x = BatchNormalization()(x)
  
    outputs = Conv2D(filters = 1, kernel_size = (1,1), activation = "sigmoid")(x)

    return Model(inputs=[inputs], outputs = [outputs])

In [ ]:
epochs=50

In [ ]:
for i in range(5):
    filepath = f"G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\simplenet_8-{i+1}.keras"

    checkpoint = ModelCheckpoint(filepath, 
                             verbose=1 
                             )
    
    simplenet_8 = simple_net_8(input_shape = (None,None,3))
    simplenet_8.compile(optimizer='adam', loss=dice_coeff_loss, metrics=[dice_coeff])
    
    history = simplenet_8.fit(
        train_generator,
        steps_per_epoch=len(df_train) // batch_size,
        epochs=epochs,
        callbacks=[checkpoint],
        validation_data=test_generator,
        validation_steps=len(df_test) // batch_size
    )
    
    simplenet_8_dice,simplenet_8_thin_dice,simplenet_8_thick_dice = evaluate(simplenet_8,512,None,None,None)

In [ ]:
simplenet_8_dice

In [ ]:
simplenet_8_thin_dice

In [ ]:
simplenet_8_thick_dice

# DeepLabv3+

In [ ]:
import DIFF_MODELS_2D as deeplab

In [ ]:
epochs=50

In [ ]:
for i in range(5):
    filepath = f"G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\deeplab-{i+1}.keras"

    checkpoint = ModelCheckpoint(filepath, 
                             verbose=1 
                             )
    
    deeplab_model = deeplab.deeplabv3_plus()
    ADAM = Adam(learning_rate=0.001, clipnorm=1., clipvalue=0.5)  
    deeplab_model.compile(optimizer=ADAM, loss=dice_coeff_loss, metrics=[dice_coeff])
    
    history = deeplab_model.fit(
        train_generator,
        steps_per_epoch=len(df_train) // batch_size,
        epochs=epochs,
        callbacks=[checkpoint],
        validation_data=test_generator,
        validation_steps=len(df_test) // batch_size
    )
    
    deeplab_dice,deeplab_thin_dice,deeplab_thick_dice = evaluate(deeplab_model,512,None,None,None)

In [ ]:
deeplab_dice

In [ ]:
deeplab_thin_dice

In [ ]:
deeplab_thick_dice

# Pocketnet (Filters = 16)

In [ ]:
pocketnet_16 = models.att_unet_2d((None,None,3), filter_num = [16,16,16,16],
                                       n_labels = 1,
                                       stack_num_down = 2, stack_num_up = 2,
                                       activation = 'ReLU',
                                       output_activation = 'Sigmoid',
                                       atten_activation='ReLU', attention='add',
                                       pool=True, unpool=True,
                                       batch_norm = True)

In [ ]:
pocketnet_16.summary()

In [ ]:
epochs = 50

In [ ]:
for i in range(5):
    pocketnet_16 = models.att_unet_2d((None,None,3), filter_num = [16,16,16,16],
                                       n_labels = 1,
                                       stack_num_down = 2, stack_num_up = 2,
                                       activation = 'ReLU',
                                       output_activation = 'Sigmoid',
                                       atten_activation='ReLU', attention='add',
                                       pool=True, unpool=True,
                                       batch_norm = True)
    
    filepath = f"G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\pocketnet_16-{i+1}.keras"

    checkpoint = ModelCheckpoint(filepath, 
                             verbose=1 
                             )
    
    pocketnet_16.compile(optimizer='adam', loss=dice_coeff_loss, metrics=[dice_coeff])
    
    history = pocketnet_16.fit(
        train_generator,
        steps_per_epoch=len(df_train) // batch_size,
        epochs=epochs,
        callbacks=[checkpoint],
        validation_data=test_generator,
        validation_steps=len(df_test) // batch_size
    )
    
    pocketnet_16_dice,pocketnet_16_thin_dice,pocketnet_16_thick_dice = evaluate(pocketnet_16,512,None,None,None)

In [ ]:
np.mean(pocketnet_16_dice)

In [ ]:
np.mean(pocketnet_16_thin_dice)

In [ ]:
np.mean(pocketnet_16_thick_dice)

# Resnet Type Net (Filters=16)

In [ ]:
def encoder_block(inputs, filters):
    conv = Conv2D(filters, kernel_size = (3,3), padding="same", activation = 'relu')(inputs)
    conv = Conv2D(filters, kernel_size = (3,3), padding="same", activation = 'relu')(conv)
    pool = MaxPooling2D(pool_size=(2,2))(conv)

    return pool,conv

def decoder_block(inputs, filters, x):
    up = Conv2DTranspose(filters, kernel_size=(2, 2), strides=(2, 2), padding="same")(inputs)
    conv = Conv2D(filters, kernel_size = (3,3), padding="same", activation = 'relu')(up)
    conv = Conv2D(filters, kernel_size = (3,3), padding="same", activation = 'relu')(conv)

    if x is not None:
        x = BatchNormalization()(x)
        conv = BatchNormalization()(conv)
        conv = add([x, conv])

    return conv

In [ ]:
def u_block(inputs, filters):
    b1,c1 = encoder_block(inputs, filters)
    b2,c2 = encoder_block(b1, filters)

    conv = Conv2D(filters, kernel_size = (3,3), padding="same", activation = 'relu')(b2)
    conv = Conv2D(filters, kernel_size = (3,3), padding="same", activation = 'relu')(conv)

    b4 = decoder_block(conv, filters,None)
    b5 = decoder_block(b4, filters, c1)

    return b5

In [ ]:
f = 16
num_blocks = 4

In [ ]:
def res_type_16(input_shape = (None, None, 3)):

    inputs = Input(input_shape)
    x = Conv2D(filters = f, kernel_size = (3,3), padding = 'same', activation = 'relu')(inputs)
    x = BatchNormalization()(x)

    for i in range(num_blocks):
        x = u_block(x,f)
        b = Conv2D(filters = f, kernel_size = (3,3), padding = 'same', activation = 'relu')(inputs)
        b = BatchNormalization()(b)
        x = add([x, b])
    
    outputs = Conv2D(filters = 1, kernel_size = (1,1), activation = "sigmoid")(x)
    
    return Model(inputs=[inputs], outputs = [outputs])

In [ ]:
epochs=50

In [ ]:
for i in range(5):
    filepath = f"G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\restype_16-{i+1}.keras"

    checkpoint = ModelCheckpoint(filepath, 
                             verbose=1 
                             )
    
    restype_16 = res_type_16(input_shape = (None,None,3))
    restype_16.compile(optimizer='adam', loss=dice_coeff_loss, metrics=[dice_coeff])
    
    history = restype_16.fit(
        train_generator,
        steps_per_epoch=len(df_train) // batch_size,
        epochs=epochs,
        callbacks=[checkpoint],
        validation_data=test_generator,
        validation_steps=len(df_test) // batch_size
    )
    
    restype_16_dice,restype_16_thin_dice,restype_16_thick_dice = evaluate(restype_16,512,None,None,None)

In [ ]:
np.mean(restype_16_dice)

In [ ]:
np.mean(restype_16_thin_dice)

In [ ]:
np.mean(restype_16_thick_dice)

# MyNet (Filters=32)

In [ ]:
f = 32
num_blocks = 8

In [ ]:
def my_net_32(input_shape = (None, None, 3)):
    inputs = Input(input_shape)
    x = Conv2D(filters = f, kernel_size = (3,3), padding = 'same', activation = 'relu')(inputs)
    x = BatchNormalization()(x)
    
    for i in range(num_blocks):
        x = block(x,f)
        
        if i == 3 or i == 7:
            x = BatchNormalization()(x)
            b = Conv2D(filters = f, kernel_size = (3,3), padding = 'same', activation = 'relu')(inputs)
            b = BatchNormalization()(b)
            x = add([x, b])
  
    outputs = Conv2D(filters = 1, kernel_size = (1,1), activation = "sigmoid")(x)

    return Model(inputs=[inputs], outputs = [outputs])

In [ ]:
epochs=50

In [ ]:
for i in range(5):
    filepath = f"G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\mynet_32-{i+1}.keras"

    checkpoint = ModelCheckpoint(filepath, 
                             verbose=1 
                             )
    
    mynet_32 = my_net_32(input_shape = (None,None,3))
    mynet_32.compile(optimizer='adam', loss=dice_coeff_loss, metrics=[dice_coeff])
    
    history = mynet_32.fit(
        train_generator,
        steps_per_epoch=len(df_train) // batch_size,
        epochs=epochs,
        callbacks=[checkpoint],
        validation_data=test_generator,
        validation_steps=len(df_test) // batch_size
    )
    
    mynet_32_dice,mynet_32_thin_dice,mynet_32_thick_dice = evaluate(mynet_32,512,None,None,None)

In [ ]:
np.mean(mynet_32_dice)

In [ ]:
np.mean(mynet_32_thin_dice)

In [ ]:
np.mean(mynet_32_thick_dice)

# MyNet (Filters=16) 16 blocks

In [ ]:
f = 16
num_blocks = 16

In [ ]:
def my_net_16_16b(input_shape = (None, None, 3)):
    inputs = Input(input_shape)
    x = Conv2D(filters = f, kernel_size = (3,3), padding = 'same', activation = 'relu')(inputs)
    x = BatchNormalization()(x)
    
    for i in range(num_blocks):
        x = block(x,f)
        
        if i == 7 or i == 11 or i == 15:
            x = BatchNormalization()(x)
            b = Conv2D(filters = f, kernel_size = (3,3), padding = 'same', activation = 'relu')(inputs)
            b = BatchNormalization()(b)
            x = add([x, b])
  
    outputs = Conv2D(filters = 1, kernel_size = (1,1), activation = "sigmoid")(x)

    return Model(inputs=[inputs], outputs = [outputs])

In [ ]:
epochs=50

In [ ]:
for i in range(5):
    filepath = f"G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\mynet_16_16b-{i+1}.keras"

    checkpoint = ModelCheckpoint(filepath, 
                             verbose=1 
                             )
    
    mynet_16_16b = my_net_16_16b(input_shape = (None,None,3))
    mynet_16_16b.compile(optimizer='adam', loss=dice_coeff_loss, metrics=[dice_coeff])
    
    history = mynet_16_16b.fit(
        train_generator,
        steps_per_epoch=len(df_train) // batch_size,
        epochs=epochs,
        callbacks=[checkpoint],
        validation_data=test_generator,
        validation_steps=len(df_test) // batch_size
    )
    
    mynet_16_16b_dice,mynet_16_16b_thin_dice,mynet_16_16b_thick_dice = evaluate(mynet_16_16b,512,None,None,None)

In [ ]:
np.mean(mynet_16_16b_dice)

In [ ]:
np.mean(mynet_16_16b_thin_dice)

In [ ]:
np.mean(mynet_16_16b_thick_dice)

# MyNet (Filters=8) 16 blocks

In [ ]:
f = 8
num_blocks = 16

In [ ]:
def my_net_8_16b(input_shape = (None, None, 3)):
    inputs = Input(input_shape)
    x = Conv2D(filters = f, kernel_size = (3,3), padding = 'same', activation = 'relu')(inputs)
    x = BatchNormalization()(x)
    
    for i in range(num_blocks):
        x = block(x,f)
        
        if i == 7 or i == 11 or i == 15:
            x = BatchNormalization()(x)
            b = Conv2D(filters = f, kernel_size = (3,3), padding = 'same', activation = 'relu')(inputs)
            b = BatchNormalization()(b)
            x = add([x, b])
  
    outputs = Conv2D(filters = 1, kernel_size = (1,1), activation = "sigmoid")(x)

    return Model(inputs=[inputs], outputs = [outputs])

In [ ]:
epochs=50

In [ ]:
for i in range(5):
    filepath = f"G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\mynet_8_16b-{i+1}.keras"

    checkpoint = ModelCheckpoint(filepath, 
                             verbose=1 
                             )
    
    mynet_8_16b = my_net_8_16b(input_shape = (None,None,3))
    mynet_8_16b.compile(optimizer='adam', loss=dice_coeff_loss, metrics=[dice_coeff])
    
    history = mynet_8_16b.fit(
        train_generator,
        steps_per_epoch=len(df_train) // batch_size,
        epochs=epochs,
        callbacks=[checkpoint],
        validation_data=test_generator,
        validation_steps=len(df_test) // batch_size
    )
    
    mynet_8_16b_dice,mynet_8_16b_thin_dice,mynet_8_16b_thick_dice = evaluate(mynet_8_16b,512,None,None,None)

In [ ]:
np.mean(mynet_8_16b_dice)

In [ ]:
np.mean(mynet_8_16b_thin_dice)

In [ ]:
np.mean(mynet_8_16b_thick_dice)

# Simple Net (Filters=32)

In [ ]:
f = 32
num_blocks = 8

In [ ]:
def simple_net_32(input_shape = (None, None, 3)):
    inputs = Input(input_shape)
    x = Conv2D(filters = f, kernel_size = (3,3), padding = 'same', activation = 'relu')(inputs)
    x = BatchNormalization()(x)
    
    for i in range(num_blocks):
        x = block(x,f)
        
        if i == 3 or i == 7:
            x = BatchNormalization()(x)
  
    outputs = Conv2D(filters = 1, kernel_size = (1,1), activation = "sigmoid")(x)

    return Model(inputs=[inputs], outputs = [outputs])

In [ ]:
epochs=50

In [ ]:
for i in range(5):
    filepath = f"G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\simplenet_32-{i+1}.keras"

    checkpoint = ModelCheckpoint(filepath, 
                             verbose=1 
                             )
    
    simplenet_32 = simple_net_32(input_shape = (None,None,3))
    simplenet_32.compile(optimizer='adam', loss=dice_coeff_loss, metrics=[dice_coeff])
    
    history = simplenet_32.fit(
        train_generator,
        steps_per_epoch=len(df_train) // batch_size,
        epochs=epochs,
        callbacks=[checkpoint],
        validation_data=test_generator,
        validation_steps=len(df_test) // batch_size
    )
    
    simplenet_32_dice,simplenet_32_thin_dice,simplenet_32_thick_dice = evaluate(simplenet_32,512,None,None,None)

In [ ]:
np.mean(simplenet_32_dice)

In [ ]:
np.mean(simplenet_32_thin_dice)

In [ ]:
np.mean(simplenet_32_thick_dice)

# Simple Net (Filters=16) 16 blocks

In [ ]:
f = 16
num_blocks = 16

In [ ]:
def simple_net_16_16b(input_shape = (None, None, 3)):
    inputs = Input(input_shape)
    x = Conv2D(filters = f, kernel_size = (3,3), padding = 'same', activation = 'relu')(inputs)
    x = BatchNormalization()(x)
    
    for i in range(num_blocks):
        x = block(x,f)
        
        if i == 7 or i == 11 or i == 15:
            x = BatchNormalization()(x)
  
    outputs = Conv2D(filters = 1, kernel_size = (1,1), activation = "sigmoid")(x)

    return Model(inputs=[inputs], outputs = [outputs])

In [ ]:
epochs=50

In [ ]:
for i in range(5):
    filepath = f"G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\simplenet_16_16b-{i+1}.keras"

    checkpoint = ModelCheckpoint(filepath, 
                             verbose=1 
                             )
    
    simplenet_16_16b = simple_net_16_16b(input_shape = (None,None,3))
    simplenet_16_16b.compile(optimizer='adam', loss=dice_coeff_loss, metrics=[dice_coeff])
    
    history = simplenet_16_16b.fit(
        train_generator,
        steps_per_epoch=len(df_train) // batch_size,
        epochs=epochs,
        callbacks=[checkpoint],
        validation_data=test_generator,
        validation_steps=len(df_test) // batch_size
    )
    
    simplenet_16_16b_dice,simplenet_16_16b_thin_dice,simplenet_16_16b_thick_dice = evaluate(simplenet_16_16b,512,None,None,None)

In [ ]:
np.mean(simplenet_16_16b_dice)

In [ ]:
np.mean(simplenet_16_16b_thin_dice)

In [ ]:
np.mean(simplenet_16_16b_thick_dice)

# Simple Net (Filters=8) 16 blocks

In [ ]:
f = 8
num_blocks = 16

In [ ]:
def simple_net_8_16b(input_shape = (None, None, 3)):
    inputs = Input(input_shape)
    x = Conv2D(filters = f, kernel_size = (3,3), padding = 'same', activation = 'relu')(inputs)
    x = BatchNormalization()(x)
    
    for i in range(num_blocks):
        x = block(x,f)
        
        if i == 7 or i == 11 or i == 15:
            x = BatchNormalization()(x)
  
    outputs = Conv2D(filters = 1, kernel_size = (1,1), activation = "sigmoid")(x)

    return Model(inputs=[inputs], outputs = [outputs])

In [ ]:
epochs=50

In [ ]:
for i in range(5):
    filepath = f"G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\simplenet_8_16b-{i+1}.keras"

    checkpoint = ModelCheckpoint(filepath, 
                             verbose=1 
                             )
    
    simplenet_8_16b = simple_net_8_16b(input_shape = (None,None,3))
    simplenet_8_16b.compile(optimizer='adam', loss=dice_coeff_loss, metrics=[dice_coeff])
    
    history = simplenet_8_16b.fit(
        train_generator,
        steps_per_epoch=len(df_train) // batch_size,
        epochs=epochs,
        callbacks=[checkpoint],
        validation_data=test_generator,
        validation_steps=len(df_test) // batch_size
    )
    
    simplenet_8_16b_dice,simplenet_8_16b_thin_dice,simplenet_8_16b_thick_dice = evaluate(simplenet_8_16b,512,None,None,None)

In [ ]:
np.mean(simplenet_8_16b_dice)

In [ ]:
np.mean(simplenet_8_16b_thin_dice)

In [ ]:
np.mean(simplenet_8_16b_thick_dice)

# Separable Pocket Net

In [12]:
def encoder_block(inputs, filters):
    conv = SeparableConv2D(filters, kernel_size = (3,3), padding="same", activation = 'relu')(inputs)
    conv = SeparableConv2D(filters, kernel_size = (3,3), padding="same", activation = 'relu')(conv)
    x = BatchNormalization()(conv)
    pool = MaxPooling2D(pool_size=(2,2))(x)
    
    return pool,x

def decoder_block(inputs, filters, x):
    conv = SeparableConv2D(filters, kernel_size = (3,3), padding="same", activation = 'relu')(inputs)
    up = UpSampling2D((2, 2),interpolation='bilinear')(conv)
    con = add([up, x])
    conv = SeparableConv2D(filters, kernel_size = (3,3), padding="same", activation = 'relu')(con)
    conv = SeparableConv2D(filters, kernel_size = (3,3), padding="same", activation = 'relu')(conv)
    x = BatchNormalization()(conv)
    
    return x

In [13]:
f = 16

In [14]:
def Separable_16(input_shape = (None, None, 3)):
    inputs = Input(input_shape)
    x = SeparableConv2D(filters = f, kernel_size = (3,3), padding = 'same', activation = 'relu')(inputs)
    x = BatchNormalization()(x)
    
    b1,c1 = encoder_block(x, f)
    b2,c2 = encoder_block(b1, f*2)
    b3,c3 = encoder_block(b2, f*4)
    b4,c4 = encoder_block(b3, f*8)

    conv = SeparableConv2D(f*8, kernel_size = (3,3), padding="same", activation = 'relu')(b4)
    conv = SeparableConv2D(f*8, kernel_size = (3,3), padding="same", activation = 'relu')(conv)

    b4 = decoder_block(conv, f*8, c4)
    b5 = decoder_block(b4, f*4, c3)
    b6 = decoder_block(b5, f*2, c2)
    b7 = decoder_block(b6, f, c1)
    
    outputs = Conv2D(filters = 1, kernel_size = (1,1), activation = "sigmoid")(b7)
    
    return Model(inputs=[inputs], outputs = [outputs])

In [15]:
filepath = f"G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\separable_upsample_16.keras"

checkpoint = ModelCheckpoint(filepath, 
                             verbose=1 
                             )

In [16]:
separable_16 = Separable_16(input_shape = (None,None,3))
separable_16.compile(optimizer='adam', loss=dice_coeff_loss, metrics=[dice_coeff])

In [17]:
epochs = 50

In [18]:
history = separable_16.fit(
        train_generator,
        steps_per_epoch=len(df_train) // batch_size,
        epochs=epochs,
        callbacks=[checkpoint],
        validation_data=test_generator,
        validation_steps=len(df_test) // batch_size
    )

Epoch 1/50
75/75 [==============================] - ETA: 0s - loss: 0.7843 - dice_coeff: 0.2157
Epoch 1: saving model to G:\Harsh_Birva_minor_Proj\new_retina_models_5\separable_upsample_16.keras
75/75 [==============================] - 111s 1s/step - loss: 0.7843 - dice_coeff: 0.2157 - val_loss: 0.8635 - val_dice_coeff: 0.1365
Epoch 2/50
75/75 [==============================] - ETA: 0s - loss: 0.5752 - dice_coeff: 0.4248
Epoch 2: saving model to G:\Harsh_Birva_minor_Proj\new_retina_models_5\separable_upsample_16.keras
75/75 [==============================] - 109s 1s/step - loss: 0.5752 - dice_coeff: 0.4248 - val_loss: 0.9073 - val_dice_coeff: 0.0927
Epoch 3/50
75/75 [==============================] - ETA: 0s - loss: 0.3458 - dice_coeff: 0.6542
Epoch 3: saving model to G:\Harsh_Birva_minor_Proj\new_retina_models_5\separable_upsample_16.keras
75/75 [==============================] - 120s 2s/step - loss: 0.3458 - dice_coeff: 0.6542 - val_loss: 0.9579 - val_dice_coeff: 0.0421
Epoch 4/50
75

Epoch 26/50
75/75 [==============================] - ETA: 0s - loss: 0.0995 - dice_coeff: 0.9005
Epoch 26: saving model to G:\Harsh_Birva_minor_Proj\new_retina_models_5\separable_upsample_16.keras
75/75 [==============================] - 119s 2s/step - loss: 0.0995 - dice_coeff: 0.9005 - val_loss: 0.1187 - val_dice_coeff: 0.8813
Epoch 27/50
75/75 [==============================] - ETA: 0s - loss: 0.0975 - dice_coeff: 0.9025
Epoch 27: saving model to G:\Harsh_Birva_minor_Proj\new_retina_models_5\separable_upsample_16.keras
75/75 [==============================] - 105s 1s/step - loss: 0.0975 - dice_coeff: 0.9025 - val_loss: 0.1184 - val_dice_coeff: 0.8816
Epoch 28/50
75/75 [==============================] - ETA: 0s - loss: 0.0963 - dice_coeff: 0.9037
Epoch 28: saving model to G:\Harsh_Birva_minor_Proj\new_retina_models_5\separable_upsample_16.keras
75/75 [==============================] - 105s 1s/step - loss: 0.0963 - dice_coeff: 0.9037 - val_loss: 0.1261 - val_dice_coeff: 0.8739
Epoch 2